#MISC

In [ ]:
from pyVHR.plot.visualize import *

In [ ]:
from scipy.ndimage import gaussian_filter
import mediapipe as mp
from sklearn.metrics import euclidean_distances
import cv2
import numpy as np
# compute the euclidean distances between all landmarks
image_file_name ="/home/devel/pyVHR_pack/example.jpg"
landmarks_list =  [_ for _ in range(468)]
imag = cv2.imread(image_file_name,cv2.COLOR_RGB2BGR)
imag = cv2.cvtColor(imag, cv2.COLOR_BGR2RGB)
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate([image_file_name,]):
    image = cv2.imread(file)
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
      continue
    width = image.shape[1]
    height = image.shape[0]
    face_landmarks = results.multi_face_landmarks[0]
    ldmks = np.zeros((468, 3), dtype=np.float32)
    for idx, landmark in enumerate(face_landmarks.landmark):
        if ((landmark.HasField('visibility') and landmark.visibility < VISIBILITY_THRESHOLD)
                or (landmark.HasField('presence') and landmark.presence < PRESENCE_THRESHOLD)):
            ldmks[idx, 0] = -1.0
            ldmks[idx, 1] = -1.0
            ldmks[idx, 2] = -1.0
        else:
            coords = mp_drawing._normalized_to_pixel_coordinates(
                landmark.x, landmark.y, width, height)
            if coords:
                ldmks[idx, 0] = coords[1]
                ldmks[idx, 1] = coords[0]
                ldmks[idx, 2] = idx
            else:
                ldmks[idx, 0] = -1.0
                ldmks[idx, 1] = -1.0
                ldmks[idx, 2] = -1.0
euclid_distances = euclidean_distances(ldmks[:,0:2], ldmks[:,0:2])

In [ ]:
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
from numba import njit, prange
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import euclidean_distances

@njit(parallel=True)
def __sum_images(a,b,alpha):
  for x in prange(a.shape[0]):
    for y in prange(a.shape[1]):
      a[x,y,0] = min((1-alpha)*a[x,y,0] + alpha*b[x,y,0],255)
      a[x,y,1] = min((1-alpha)*a[x,y,1] + alpha*b[x,y,1],255)
      a[x,y,2] = min((1-alpha)*a[x,y,2] + alpha*b[x,y,2],255)
  return a

def plot_heat_map(err, err_sorted, radius, sigma, alpha, reverse=True):
  configure_plotly_browser_state()
  image_file_name = "/home/devel/pyVHR_pack/example.jpg"
  imag = cv2.imread(image_file_name,cv2.COLOR_RGB2BGR)
  imag = cv2.cvtColor(imag, cv2.COLOR_BGR2RGB)
  width = imag.shape[1]
  height = imag.shape[0]
  to_keep = [err_sorted[0],]
  for e in err_sorted[1:]:
    if np.min(euclid_distances[e,to_keep]) > radius:
      to_keep.append(e)
  to_keep = list(set(to_keep))
  filtered_ldmks = []
  for idx in to_keep:
      filtered_ldmks.append([ldmks[idx,0],ldmks[idx,1],ldmks[idx,2]])
  filtered_ldmks = np.array(filtered_ldmks, dtype=np.float32)
  grid = np.zeros((height, width,1),dtype=np.float32)
  _err = err
  if reverse: 
    _err = 1.0 / err
  for e in filtered_ldmks:
    grid[int(e[0]),int(e[1]),0] = _err[int(e[2])]
  Z = gaussian_filter(grid, sigma=sigma)
  Z = (Z/np.max(Z)) * 255
  Z = Z.astype(np.uint8)
  heat_map = cv2.applyColorMap(Z[:,:,0],cv2.COLORMAP_RAINBOW) 
  __sum_images(imag, heat_map, alpha)
  fig = px.imshow(imag)
  for l in filtered_ldmks:
      name = 'landmark_' + str(int(l[2]))
      fig.add_trace(go.Scatter(x=(l[1],), y=(l[0],),mode='markers',marker_color='rgba(0,0,255,1.0)', marker=dict(color='LightSkyBlue',size=2,),name=name))
  fig.show()

# HEAT MAP 

In [ ]:
# load pickle files that contain np.ndarray with shape (num_landmarks, errors), where errors are 0:RMSE, 1:MAE, 2:MAX, 3:PCC
import pickle
import numpy as np
import cv2
import os
to_open= []
path = "/home/devel/pyVHR_pack/results/heat_maps/lgi/"
for filename in os.listdir(path):
  to_open.append(str(path+filename))

errors = []
for e in to_open:
  errors.append(pickle.load(open(e,'rb')))

errors =np.array(errors)
RMSE_errors =  errors[:,:,0]
MAE_errors =  errors[:,:,1]
MAX_errors =  errors[:,:,2]
PCC_errors =  errors[:,:,3]

In [ ]:
# RMSE errors
RMSE_errors = RMSE_errors/np.sum(RMSE_errors)
RMSE_errors = np.sum(RMSE_errors,axis=0)
RMSE_errors = RMSE_errors/np.max(RMSE_errors)
print(RMSE_errors.shape)
print(np.min(RMSE_errors))
print(np.max(RMSE_errors))

In [ ]:
# MAE errors
MAE_errors = MAE_errors/np.sum(MAE_errors)
MAE_errors = np.sum(MAE_errors,axis=0)
MAE_errors = MAE_errors/np.max(MAE_errors)
print(MAE_errors.shape)
print(np.min(MAE_errors))
print(np.max(MAE_errors))

In [ ]:
# MAX errors
MAX_errors = MAX_errors/np.sum(MAX_errors)
MAX_errors = np.sum(MAX_errors,axis=0)
MAX_errors = MAX_errors/np.max(MAX_errors)
print(MAX_errors.shape)
print(np.min(MAX_errors))
print(np.max(MAX_errors))

In [ ]:
# PCC errors
PCC_errors = PCC_errors+1.0
PCC_errors = PCC_errors/np.sum(PCC_errors)
PCC_errors = np.sum(PCC_errors,axis=0)
PCC_errors = PCC_errors/np.max(PCC_errors)
print(PCC_errors.shape)
print(np.min(PCC_errors))
print(np.max(PCC_errors))

In [ ]:
top_n = 50
radius = 40*2

In [ ]:
# RMSE 
RMSE_sorted_errors = np.argsort(RMSE_errors)
print(RMSE_sorted_errors[:top_n])
print(RMSE_errors[RMSE_sorted_errors[:top_n]])

In [ ]:
visualize_landmarks_list("/home/devel/pyVHR_pack/example.jpg", RMSE_sorted_errors[:top_n])

In [ ]:
plot_heat_map(RMSE_errors,RMSE_sorted_errors, radius, radius, 0.5)

In [ ]:
# MAE
MAE_sorted_errors = np.argsort(MAE_errors)
print(MAE_sorted_errors[:100])
print(MAE_errors[MAE_sorted_errors[:top_n]])

In [ ]:
visualize_landmarks_list("/home/devel/pyVHR_pack/example.jpg", MAE_sorted_errors[:top_n])

In [ ]:
plot_heat_map(MAE_errors,MAE_sorted_errors, radius, radius, 0.5)

In [ ]:
# MAX
MAX_sorted_errors = np.argsort(MAX_errors)
print(MAX_sorted_errors[:top_n])
print(MAX_errors[MAX_sorted_errors[:top_n]])

In [ ]:
visualize_landmarks_list("/home/devel/pyVHR_pack/example.jpg", MAX_sorted_errors[:top_n])

In [ ]:
plot_heat_map(MAX_errors,MAX_sorted_errors, radius,radius, 0.5)

In [ ]:
# PCC
PCC_sorted_errors = np.argsort(PCC_errors)[::-1]
print(PCC_sorted_errors.shape)
print(PCC_sorted_errors[:top_n])
print(PCC_errors[PCC_sorted_errors[:top_n]])

In [ ]:
visualize_landmarks_list("/home/devel/pyVHR_pack/example.jpg", PCC_sorted_errors[:top_n])

In [ ]:
plot_heat_map(PCC_errors,PCC_sorted_errors, radius, radius, 0.5, reverse=False)

In [ ]:
# BEST landmarks intersection

best_rmse = RMSE_sorted_errors[:top_n]
best_mae = MAE_sorted_errors[:top_n]
best_max = MAX_sorted_errors[:top_n]
best_pcc = PCC_sorted_errors[:top_n]
top_intersection = set(best_rmse).intersection(set(best_mae).intersection(set(best_max).intersection(best_pcc)))
print(top_intersection)
print(len(top_intersection))
visualize_landmarks_list("/home/devel/pyVHR_pack/example.jpg", list(top_intersection))